# Week 09 Group 40

In [2]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import random

#Generate our Dataset


In [3]:
class gen_class(object):


    def __init__(self, n ):
        self.seq_length = n
        self.dataset_length = 0
        self.sequence_to_choose_from = range(1,10)
        self.seq = []


    def __iter__(self):
        return self


    def __next__(self):
        return self.next()

    def next(self):
        if self.dataset_length < 64*64: 
            self.dataset_length +=1
            q1 = random.choice(self.sequence_to_choose_from)
            q2 = random.choice(self.sequence_to_choose_from)
            # sample until the queried digits arent the same
            while (q1 == q2):
              q2 = random.choice(self.sequence_to_choose_from)
            self.seq = []
            for _ in range(self.seq_length):
              self.seq.append([q1,q2,random.choice(self.sequence_to_choose_from)])
            #append relevant queries
            
            target1 = self.seq.count([q1,q2,q1])
            target2 = self.seq.count([q1,q2,q2])
            if(target1 < target2):
                target = [target2]
            elif(target2 < target1):
                target = [target1]
            elif(target1 == target2):
                target = [0]
            return  (tf.reshape(self.seq, [25,3]), target)
        raise StopIteration()

In [4]:
#spec = tf.TensorSpec(([25,3],1), tf.dtypes.float32)
#ds = tf.data.Dataset.from_generator(generator=gen_class, output_signature= spec, args= [25])
ds = tf.data.Dataset.from_generator(generator=gen_class, output_shapes=([25,3],None) ,output_types=(tf.dtypes.float32, tf.dtypes.float32), args= [25])
print(ds)

<FlatMapDataset shapes: ((25, 3), <unknown>), types: (tf.float32, tf.float32)>


In [5]:
#ds = (ds.cache().shuffle(buffer_size = 10000, reshuffle_each_iteration=True).batch(64, )).prefetch(tf.data.experimental.AUTOTUNE)
ds = (ds.cache().shuffle(buffer_size = 10000, reshuffle_each_iteration=True)).prefetch(tf.data.experimental.AUTOTUNE)

print(ds)

<PrefetchDataset shapes: ((25, 3), <unknown>), types: (tf.float32, tf.float32)>


In [6]:
# for e in ds.take(1):
#     print(e)

#LSTM implementation

In [12]:
class LSTM_Cell(tf.keras.Model):
    def __init__(self):
        super(LSTM_Cell, self).__init__()
        self.input_shape_ = (25,3)
        self.output_shape_ = ()
        self.hidden_state = tf.zeros([1,1],tf.dtypes.float32)
        self.cell_state = tf.zeros([4,1],tf.dtypes.float32)
        self.forget_gate = tf.keras.layers.Dense(1, activation = "sigmoid")
        self.input_gate = tf.keras.layers.Dense(4, activation = "sigmoid")
        self.cell_state_candidates = tf.keras.layers.Dense(1, activation = "tanh")
        self.output_gate = tf.keras.layers.Dense(1, activation = "sigmoid")
        
    def __call__(self,input, Training = True):
        # concatenate the input of our current timestep with our current hidden state
        #print(len(input))
        self.hidden_state = tf.zeros([1,1],tf.dtypes.float32)
        for timestep in range(len(input)):
            
            #print(input)
            #x_t = input[:,timestep,:]
            x_t = tf.reshape(input[timestep,:],[1,3])
            print("x_t = " + str(x_t))
            #use when batched
            #concat_input = tf.concat([input, self.hidden_state], axis = 2)

            concat_input = tf.concat([x_t, self.hidden_state], axis = -1)
            print("concat_input before reshape = " + str(concat_input))
            concat_input = tf.reshape(concat_input, (4,1))
            print("concat_input after reshape = " + str(concat_input))
            #print(concat_input)
            # calculate the forget, input and output filter aswell as our new cell-state-candidates based on the concatenated input
            forget_filter = self.forget_gate(concat_input)
            print("forget filter = " + str(forget_filter))
            #forget_filter = tf.transpose(forget_filter)
            input_filter = self.input_gate(concat_input)
            print("input filter = " + str(input_filter))
            #input_filter = tf.transpose(input_filter)
            candidates = self.cell_state_candidates(concat_input)
            print("candidates = " + str(candidates))
            output_filter = self.output_gate(concat_input)
            print("output filter = " + str(output_filter))

            #calculate new cellstate from our filters and old cellstate
            #print(self.cell_state)
            
            input_filter = tf.transpose(input_filter)
            #print(input_filter)
            #print(candidates)
            #print(tf.matmul(input_filter, candidates))
                                    #(4,1)                                                  (4,1)
            self.cell_state = tf.matmul(tf.transpose(forget_filter), self.cell_state) + tf.matmul(input_filter, candidates)
            print("cell state = " + str(self.cell_state))
            #calculate new hidden-state/output based on our new cellstate and output_filter
                    #(1,1)              (1,4)                                 (4,1)
            self.hidden_state = tf.matmul(tf.transpose(output_filter), tf.keras.activations.tanh(self.cell_state))
            print("hidden state = " + str(self.hidden_state))
            print()
            print()
        #print(input)
        print("loop ende")
        print()
        print()
        return self.hidden_state


In [8]:
lstm = LSTM_Cell()

In [ ]:
gen = gen_class(25)
for e,y in ds.take(2):
    print(e)
    print(lstm(e))

## training

In [10]:
def train_step(model, input, target, loss_function, optimizer, training = True):
    with tf.GradientTape() as tape:
        expectation = model(input, training)
        train_accuracy =  np.argmax(target, axis = 0) == np.argmax(expectation, axis = 0)
        loss = loss_function(target, expectation)
        gradients = tape.gradient(loss, model.trainable_variables)
        #print(target)
        #print(expectation)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss, train_accuracy

In [13]:
num_epochs = 5
learning_rate = 0.0001
running_average_factor = 0.95
# use categorical crossentropy as loss function
loss_function = tf.keras.losses.MeanSquaredError()
# use adam as optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate)
# initialize the model
model = LSTM_Cell()

# lists for plotting later
train_loss_list = []
train_accuracy_list = []

# we train for the predefined number of epochs
for step in range(num_epochs):
    # reset average for each step
    changing_average_loss = 0
    changing_average_accuracy = 0
    # train the model with the training data
    for (input, target) in ds.take(200):
        # perform training step and store the loss and accuracy of step
        train_loss, train_accuracy = train_step(model, input, target, loss_function, optimizer)
        changing_average_loss = running_average_factor * changing_average_loss + (1 - running_average_factor) * train_loss
        changing_average_accuracy = running_average_factor * changing_average_accuracy + (1 - running_average_factor) * train_accuracy
    train_loss_list.append(float(changing_average_loss))
    train_accuracy_list.append(float(changing_average_accuracy))
    
    # print values during training
    print("Epoch: " + str(step + 1))
    print("Training Accuracy: " + str(train_accuracy_list[step]))
    print("Train Loss: " + str(train_loss_list[step]))
    print()

Streaming output truncated to the last 5000 lines.
 [0.03677138]
 [0.39472005]], shape=(4, 1), dtype=float32)
cell state = tf.Tensor(
[[5.5661184e+20]
 [5.5661184e+20]
 [5.5661184e+20]
 [5.5661184e+20]], shape=(4, 1), dtype=float32)
hidden state = tf.Tensor([[0.69079125]], shape=(1, 1), dtype=float32)


x_t = tf.Tensor([[2. 9. 1.]], shape=(1, 3), dtype=float32)
concat_input before reshape = tf.Tensor([[2.         9.         1.         0.69079125]], shape=(1, 4), dtype=float32)
concat_input after reshape = tf.Tensor(
[[2.        ]
 [9.        ]
 [1.        ]
 [0.69079125]], shape=(4, 1), dtype=float32)
forget filter = tf.Tensor(
[[0.8155463 ]
 [0.9987571 ]
 [0.6777017 ]
 [0.62560713]], shape=(4, 1), dtype=float32)
input filter = tf.Tensor(
[[0.29130682 0.3040999  0.50888    0.689337  ]
 [0.01797375 0.02353788 0.5398795  0.97305375]
 [0.3906638  0.3979714  0.50444037 0.5983298 ]
 [0.4238295  0.42899913 0.50306743 0.56839055]], shape=(4, 4), dtype=float32)
candidates = tf.Tensor(
[[0.9703

KeyboardInterrupt: ignored